In [1]:
import pydss
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource, HoverTool, CustomJS
output_notebook()

Loading BokehJS ...

In [2]:
sys1 = pydss.pydss('cigre_lv.json')
s_0 = sys1.pq_3pn

In [3]:
sys1.pq_3pn = s_0
sys1.update()
sys1.pf_eval()
sys1.get_v()
sys1.get_i()

In [4]:
%timeit sys1.pf_eval()

100 loops, best of 3: 1.47 ms per loop


In [5]:
sys1.bokeh_tools()

p = figure(width=600, height=800,
           title='European LV Network (CIGRE)', 
           x_range=[-200,200], y_range=[-350, 50])

# lines:
source = ColumnDataSource(sys1.line_data)
lin = p.multi_line(source=source, xs='x_s', ys='y_s', color="red", alpha=0.5, line_width=5)

# buses:
source = ColumnDataSource(sys1.bus_data)
cr = p.circle(source=source, x='x', y='y', size=15, color="navy", alpha=0.5)

p.add_tools(HoverTool(renderers=[lin], tooltips=sys1.line_tooltip))
p.add_tools(HoverTool(renderers=[cr], tooltips=sys1.bus_tooltip))
show(p)


In [8]:
import numpy as np
from bokeh.io import output_notebook, show
from bokeh.models import ColumnDataSource
from bokeh.io import push_notebook
from bokeh.layouts import gridplot
from bokeh.plotting import figure
output_notebook()

sys1.pq_3pn = s_0
sys1.update()
sys1.pf_eval()
sys1.get_v()
sys1.get_i()

p = figure(width=600, height=300,x_range = [50,-300], y_range = [180,250])
source = ColumnDataSource(sys1.bus_data)
cr = p.circle(source=source, x='y', y='v_an', size=15, color="red", alpha=0.5)
p.circle(source=source, x='y', y='v_bn', size=15, color="green", alpha=0.5)
p.circle(source=source, x='y', y='v_cn', size=15, color="blue", alpha=0.5)
p.line([-300, 50],[231*1.05,231*1.05], color='red', line_width=5)
p.line([-300, 50],[231*0.90,231*0.90], color='blue', line_width=5)
#p.add_tools(HoverTool(renderers=[cr], tooltips=sys1.bus_tooltip))


def update(load_factor=1.0):

    sys1.pq_3pn = load_factor*s_0
    sys1.update()
    sys1.pf_eval()
    sys1.get_v()
    sys1.get_i()
    sys1.bokeh_tools()

    source.data = sys1.bus_data
    push_notebook()

#p_grid = gridplot([[p], [p_2]])
show(p, notebook_handle=True)


Loading BokehJS ...

In [9]:
from ipywidgets import interact
interact(update, load_factor=(-1,1.2, 0.01))